In [2]:
import numpy as np
import pandas as pd
from scipy.stats import t
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit

In [3]:
data = pd.read_pickle('../data/processed/feature_encoded_merged_data.pkl')
data.head()

,date,time,event_type,cleaned_event,full_text,symbol,Surprise(%),Reported EPS,Consensus EPS,hr,...,price_change_7,price_change_30,price_change_90,price_change_365,targe_price_change,prev_vix_values,dataset,target,unigram_vec,phrase_vec
0,2004-07-20,143800,EVENTS:\t\tFinancial statements and exhibits\n,[financial statements and exhibits],\n<DOCUMENT>\nFILE:FULT/FULT-8K-20040720143800...,FULT,-3.13,0.31,0.32,14.38,...,-0.50,2.57,3.01,8.21,0.414034,14.17,train,STAY,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2004-10-19,174320,EVENTS:\tResults of Operations and Financial C...,[results of operations and financial condition...,\n<DOCUMENT>\nFILE:FULT/FULT-8K-20041019174320...,FULT,0.00,0.32,0.32,17.43,...,0.16,0.39,7.58,14.93,-1.208981,15.13,train,DOWN,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2005-01-18,123338,EVENTS:\tResults of Operations and Financial C...,[results of operations and financial condition...,\n<DOCUMENT>\nFILE:FULT/FULT-8K-20050118123338...,FULT,0.00,0.33,0.33,12.33,...,0.53,0.61,5.15,13.60,-0.250990,12.47,train,STAY,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2005-04-13,140932,EVENTS:\tResults of Operations and Financial C...,[results of operations and financial condition...,\n<DOCUMENT>\nFILE:FULT/FULT-8K-20050413140932...,FULT,0.00,0.33,0.33,14.09,...,-1.75,-2.19,-1.37,8.56,0.070178,13.31,train,STAY,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,2005-07-19,132220,EVENTS:\tResults of Operations and Financial C...,[results of operations and financial condition...,\n<DOCUMENT>\nFILE:FULT/FULT-8K-20050719132220...,FULT,0.00,0.27,0.27,13.22,...,-1.28,2.88,11.54,17.17,0.604141,10.45,train,STAY,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Phrase Selection

In [4]:
t_test = pd.read_csv('../data/t-test.csv')
t_test = t_test.dropna(subset=['t values'])
t_test.shape

(2999, 4)

In [5]:
def get_prob(t_val):
    return t.sf(abs(t_val), 2998)

In [6]:
t_test['Probabilities'] = t_test['Coefficients'].apply(get_prob)

In [7]:
phrase_indicies = t_test.sort_values(by = ['Probabilities']).head(2319).index.values

In [8]:
def select_phrases(phrases):
    return np.array(phrases)[phrase_indicies]

In [9]:
%%time

data['top_phrases'] = data['phrase_vec'].apply(select_phrases)

CPU times: user 10.5 s, sys: 298 ms, total: 10.7 s
Wall time: 10.7 s


# Splits and Events

In [10]:
train = data.loc[data['dataset'] == 'train'].copy()
val = data.loc[data['dataset'] == 'val'].copy()
test = data.loc[data['dataset'] == 'test'].copy()

In [11]:
mlb = MultiLabelBinarizer()

all_events = pd.DataFrame(mlb.fit_transform(data['cleaned_event']),
                   columns = mlb.classes_,
                   index = data['cleaned_event'].index)

# Unigram Features

### Training Set

In [12]:
numerics_train = train[['Surprise(%)', 'price_change_7', 
              'price_change_30', 'price_change_90', 'price_change_365',
              'prev_vix_values']].to_numpy()
    
scale = StandardScaler()
scale.fit(numerics_train)
numerics_train = scale.transform(numerics_train)
    
train_unigrams = np.array(train['unigram_vec'].values.tolist())
train_events = all_events.iloc[train.index].to_numpy()
    
train_y = train[['target']].to_numpy().ravel()
train_X = np.concatenate((train_events, numerics_train, train_unigrams), axis = 1)

In [ ]:
[100, 10, 1.0, 0.1, 0.01]

In [13]:
uni_model = LogisticRegression(penalty = "l2", C = 10)

In [14]:
%%time

uni_model.fit(train_X, train_y)

CPU times: user 1min 1s, sys: 1min 38s, total: 2min 40s
Wall time: 1min 27s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
sum(uni_model.predict(train_X) == train_y) / len(train_y)

0.545147244184714

### Validation Set

In [16]:
numerics_val = val[['Surprise(%)', 'price_change_7', 
              'price_change_30', 'price_change_90', 'price_change_365',
              'prev_vix_values']].to_numpy()

numerics_val = scale.transform(numerics_val)
    
val_unigrams = np.array(val['unigram_vec'].values.tolist())
val_events = all_events.iloc[val.index].to_numpy()
    
val_y = val[['target']].to_numpy().ravel()
val_X = np.concatenate((val_events, numerics_val, val_unigrams), axis = 1)

In [17]:
sum(uni_model.predict(val_X) == val_y) / len(val_y)

0.4091117741565297

# Phrase Features

### Training Set

In [18]:
num_train_X = train[['Surprise(%)', 'price_change_7', 
              'price_change_30', 'price_change_90', 'price_change_365',
              'prev_vix_values']].to_numpy()

phrase_scaler = StandardScaler()
phrase_scaler.fit(num_train_X)
num_train_X = phrase_scaler.transform(num_train_X)

train_events = all_events.iloc[train.index].to_numpy()
train_phrase = np.array(train['top_phrases'].values.tolist())

phrase_train_y = train[['target']].to_numpy().ravel()

phrase_train_X = np.concatenate((train_events, num_train_X, train_phrase), axis = 1)

In [19]:
phrase_model = LogisticRegression(penalty = "l2", C = 10)

In [20]:
%%time

phrase_model.fit(phrase_train_X, phrase_train_y)

CPU times: user 1min 4s, sys: 1min 56s, total: 3min 1s
Wall time: 1min 38s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
sum(phrase_model.predict(phrase_train_X) == phrase_train_y) / len(phrase_train_y)

0.5649707803368855

### Validation Set

In [22]:
num_val_X = val[['Surprise(%)', 'price_change_7', 
              'price_change_30', 'price_change_90', 'price_change_365',
              'prev_vix_values']].to_numpy()

num_val_X = phrase_scaler.transform(num_val_X)

val_events = all_events.iloc[val.index].to_numpy()
val_phrase = np.array(val['top_phrases'].values.tolist())

phrase_val_y = val[['target']].to_numpy().ravel()
phrase_val_X = np.concatenate((val_events, num_val_X, val_phrase), axis = 1)

In [23]:
sum(phrase_model.predict(phrase_val_X) == phrase_val_y) / len(phrase_val_y)

0.38719302272205647